In [1]:
import time
import os
import struct
import numpy as np

def load_mnist_train(path):
    labels_path = os.path.join(path,'train-labels.idx1-ubyte')
    images_path = os.path.join(path, 'train-images.idx3-ubyte')
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)
    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack('>IIII',imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)
    return images, labels
def load_mnist_test(path):
    labels_path = os.path.join(path,'t10k-labels.idx1-ubyte')
    images_path = os.path.join(path, 't10k-images.idx3-ubyte')
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)
    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack('>IIII',imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)
    return images, labels
X_train,Y_train = load_mnist_train('E:\学习\大三\机器学习\MNIST')
X_test,Y_test = load_mnist_test('E:\学习\大三\机器学习\MNIST')

### svm

In [2]:
import sys
path = 'D:\LIBSVM\libsvm-3.22\python'
sys.path.append(path)
from svmutil import *
from svm  import *

y_train_svm = Y_train.tolist()
x_train_svm=[]
index = range(1,785)
for i in X_train:
    x_train_svm.append(dict(zip(index, i)))
    
y_test_svm = Y_test.tolist()
x_test_svm=[]
index = range(1,785)
for i in X_test:
    x_test_svm.append(dict(zip(index, i)))

In [3]:
def test_svm(nums,typet):
    prob  = svm_problem(y_train_svm[:nums], x_train_svm[:nums])
    param = svm_parameter('-t %d -c 4 -m 1024'%typet)
    model = svm_train(prob, param)
    p_label, p_acc, p_val = svm_predict(y_test_svm, x_test_svm, model)
    return p_acc[0],p_label

In [4]:
# 线性核
acc_svm_liner,predict_svm_liner= test_svm(10000,0)
acc_svm_liner = acc_svm_liner/100

Accuracy = 91.28% (9128/10000) (classification)


In [5]:
# 多项式核
acc_svm_polynomial,predict_svm_polynomial= test_svm(10000,1)
acc_svm_polynomial = acc_svm_polynomial/100

Accuracy = 95.77% (9577/10000) (classification)


#### 贝叶斯

In [6]:
# 高斯朴素贝叶斯
from sklearn.naive_bayes import GaussianNB
clf_g = GaussianNB()
clf_g.fit(X_train,Y_train)
predict_bayes_g = clf_g.predict(X_test)
acc_bayes_g  =sum(predict_bayes_g == Y_test)/len(Y_test)

In [7]:
# 伯努利朴素贝叶斯
from sklearn.naive_bayes import BernoulliNB
clf_b = BernoulliNB(alpha=2.0,binarize = 3.0,fit_prior=True)
clf_b.fit(X_train,Y_train)
predict_bayes_b = clf_b.predict(X_test)
acc_bayes_b = sum(predict_bayes_b == Y_test)/len(Y_test)

In [8]:
# 多项式朴素贝叶斯
from sklearn.naive_bayes import MultinomialNB
clf_m = MultinomialNB(alpha=2.0)
clf_m.fit(X_train,Y_train)
predict_bayes_m = clf_m.predict(X_test)
acc_bayes_m =sum(predict_bayes_m == Y_test)/len(Y_test)

### NN

In [9]:
def tanh(x):
    return (np.exp(x)-np.exp(-1*x))/(np.exp(x)+np.exp(-1*x))
def dtanh(y):
    return 1.0 - y*y

class NN:
    def __init__(self, ni, nh_li, no):
        # 输入层、隐藏层列表、输出层
        self.ni = ni 
        self.nh_li = nh_li
        self.nh_len = len(nh_li)
        self.no = no
        self.w = []#输入层到第一层隐含层，。。。，最后一层隐含层到输出层 总长度为 nh_len+1
        
        self.w.append(np.random.normal(0.0, pow(self.nh_li[0],-0.5), (self.nh_li[0],self.ni)))
        for i in range(self.nh_len-1):
            self.w.append(np.random.normal(0.0, pow(self.nh_li[i+1],-0.5), (self.nh_li[i+1],self.nh_li[i])))
        self.w.append(np.random.normal(0.0, pow(self.no,-0.5), (self.no,self.nh_li[self.nh_len-1])))
        
    def oneRound(self, inputs,targets,N):
        targets = np.array(targets,ndmin=2).T
        # 激活输入层
        inputs = np.array(inputs,ndmin=2).T/255
        # 隐含层输出

        outputs = []# 输入层到第一个隐含层，。。。，最后一个隐含层到输出层  总长度为 nh_len+1
        outputs.append(tanh(np.dot(self.w[0],inputs)))
        for i in range(self.nh_len-1):
            outputs.append(tanh(np.dot(self.w[i+1],outputs[i])))
        # 输出层输出
        outputs.append(tanh(np.dot(self.w[self.nh_len], outputs[self.nh_len-1])))
        
        
        deltas = []#输出层，最后一个隐含层 。。。 第一个隐含层  总长度为 nh_len+1
        #计算输出层的误差
        output_deltas = dtanh(outputs[self.nh_len])*(targets - outputs[self.nh_len])
        deltas.append(output_deltas)
        for i in range(self.nh_len):
            deltas.append(dtanh(outputs[self.nh_len-1-i])* np.dot(self.w[self.nh_len-i].T, deltas[i]))

            
        for i in range(self.nh_len):
            self.w[self.nh_len-i] += N*np.dot(deltas[i], np.transpose(outputs[self.nh_len-i-1]))
        self.w[0] += N*np.dot(deltas[self.nh_len], np.transpose(inputs))
   
        return outputs[self.nh_len]
  
    def test(self, inputs_list,Y):
        ans_lis=[]
        le = len(Y)
        ans = 0
        for i in range(le):
            inputs = np.array(inputs_list[i],ndmin=2).T/255
            outputs = []# 输入层到第一个隐含层，。。。，最后一个隐含层到输出层  总长度为 nh_len+1
            outputs.append(tanh(np.dot(self.w[0],inputs)))
            for j in range(self.nh_len-1):
                outputs.append(tanh(np.dot(self.w[j+1],outputs[j])))
            # 输出层输出
            outputs.append(tanh(np.dot(self.w[self.nh_len], outputs[self.nh_len-1])))
            if np.argmax(outputs[self.nh_len])==Y[i]:
                ans+=1
            ans_lis.append(np.argmax(outputs[self.nh_len]))
        print('准确率',ans/le)
        return ans/le,ans_lis 

    def weights(self):
        return self.w

    def train(self, X, Y,iterations=1, N=0.01):
        len_total = len(Y)
        for i in range(iterations):
            for p in range(len_total):              
                inputs = X[p]
                target = Y[p]
                targets = trans(target)
                self.oneRound(inputs,targets,N)  
def trans(li):
    re_list =[0.0]*10
    re_list[li]=1
    return re_list

In [10]:
# 三层
num=60000
n = NN(784, [89], 10)
n.train(X_train[:num],Y_train[:num],iterations=10)
acc_nn_3 ,predict_nn_3 = n.test(X_test,Y_test)

准确率 0.9479


In [11]:
# 四层
num=60000
n = NN(784, [89,50], 10)
n.train(X_train[:num],Y_train[:num],iterations=10)
acc_nn_4 ,predict_nn_4 = n.test(X_test,Y_test)

准确率 0.9699


### KNN

In [14]:
from sklearn.neighbors import KNeighborsClassifier

In [15]:
kNN_classifier_3= KNeighborsClassifier(n_neighbors=3)
kNN_classifier_3.fit(X_train, Y_train)
predict_knn_3 = kNN_classifier_3.predict(X_test)
acc_knn_3=sum(predict_knn_3 == Y_test)/len(Y_test)

In [16]:
kNN_classifier_4= KNeighborsClassifier(n_neighbors=4)
kNN_classifier_4.fit(X_train, Y_train)
predict_knn_4 = kNN_classifier_4.predict(X_test)
acc_knn_4 =sum(predict_knn_4 == Y_test)/len(Y_test)

#### 集成

In [63]:
ans = []
for i in range(10000):
    lab=[0]*10
    lab[int(predict_svm_liner[i])]+=1#0.9128
    lab[int(predict_svm_polynomial[i])]+=1#0.9577
    lab[predict_bayes_g[i]]+=1#0.5558
    lab[predict_bayes_b[i]]+=1#0.8423
    lab[predict_bayes_m[i]]+=1#0.8364
    lab[predict_knn_3[i]]+=1#0.9705
    lab[predict_knn_4[i]]+=1#0.9682
    lab[predict_nn_3[i]]+=1#  0.9479
    lab[predict_nn_4[i]]+=1#0.9696
    ans.append(lab.index(max(lab)))    
sum(ans == Y_test)/len(Y_test)

0.9642

In [96]:
ans = []
for i in range(10000):
    lab=[0]*10
    lab[int(predict_svm_liner[i])]+=acc_svm_liner
    lab[int(predict_svm_polynomial[i])]+=acc_svm_polynomial
#     lab[predict_bayes_g[i]]+=acc_bayes_g
#     lab[predict_bayes_b[i]]+=acc_bayes_b
#     lab[predict_bayes_m[i]]+=acc_bayes_m
    lab[predict_knn_3[i]]+=acc_knn_3
    lab[predict_knn_4[i]]+=acc_knn_4
    lab[predict_nn_3[i]]+=acc_nn_3
    lab[predict_nn_4[i]]+=acc_nn_4
    ans.append(lab.index(max(lab)))
sum(ans == Y_test)/len(Y_test)

0.9739

In [53]:
print(acc_svm_liner)
print(acc_svm_polynomial)
print(acc_bayes_g)
print(acc_bayes_b)
print(acc_bayes_m)
print(acc_knn_3)
print(acc_knn_4)
print(acc_nn_3)
print(acc_nn_4)

0.9128000000000001
0.9577
0.5558
0.8423
0.8364
0.9705
0.9682
0.9479
0.9699


In [64]:
ans = []
max_val=[]
for i in range(10000):
    lab=[0]*10
    lab[int(predict_svm_liner[i])]+=acc_svm_liner
    lab[int(predict_svm_polynomial[i])]+=acc_svm_polynomial
    lab[predict_bayes_g[i]]+=acc_bayes_g
    lab[predict_bayes_b[i]]+=acc_bayes_b
    lab[predict_bayes_m[i]]+=acc_bayes_m
    lab[predict_knn_3[i]]+=acc_knn_3
    lab[predict_knn_4[i]]+=acc_knn_4
    lab[predict_nn_3[i]]+=acc_nn_3
    lab[predict_nn_4[i]]+=acc_nn_4
    max_val.append(max(lab))
    ans.append(lab.index(max(lab)))
sum(ans == Y_test)/len(Y_test)

0.9666

In [93]:
ans = []
max_val=[]
for i in range(10000):
    lab=[0]*10
    lab[int(predict_svm_liner[i])]+=1#0.9128
    lab[int(predict_svm_polynomial[i])]+=1#0.9577
    lab[predict_bayes_g[i]]+=1#0.5558
    lab[predict_bayes_b[i]]+=1#0.8423
    lab[predict_bayes_m[i]]+=1#0.8364
    lab[predict_knn_3[i]]+=1#0.9705
    lab[predict_knn_4[i]]+=1#0.9682
    lab[predict_nn_3[i]]+=1#  0.9479
    lab[predict_nn_4[i]]+=1#0.9696
    ans.append(lab.index(max(lab)))    
    max_val.append(max(lab))
sum(ans == Y_test)/len(Y_test)

0.967

In [95]:
anss = 0
ansst = 0
for i in range(10000):
    if max_val[i] >=5:
        ansst+=1
        if ans[i] == Y_test[i]:
            anss+=1
print(anss/ansst)

0.9966455941168881
